In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Load the features dataset
df = pd.read_parquet('features_full.parquet')  # switch to ('features_final.parquet') if you want smaller choice set
# Define features, label, and groups
features = ['rel_TT', 'rel_transfers', 'rel_walk', 'excess_TT', 'rel_freq']
# features = [c for c in df.columns if c not in ['choice','OD','Obs_ID']] # Remove comment if you want full feature set insetad
X = df[features]

y = df['choice']
groups = df['OD'] # Switch to 'Obs_ID' if you want per-trip grouping


print("Total samples:", len(df), "Total observations:", groups.nunique())
print("Features:", features)
print("Sample X snippet:\n", X.head(5))
print("Sample y snippet:\n", y.head(5))
print("Unique groups (Obs_ID) snippet:\n", groups.head(5))

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, log_loss

# Set up grouped 5-fold cross-validation
gkf = GroupKFold(n_splits=5)
fold_metrics = []
feature_importances = []

for fold, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups=groups), start=1):
    # Split data into training and test for this fold
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    # Initialize and train Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)
    rf.fit(X_train, y_train)
    # Predict probabilities on the test set
    y_proba = rf.predict_proba(X_test)[:, 1]  # probability of class 1 (chosen)
    y_pred = rf.predict(X_test)
    # Compute metrics
    acc = accuracy_score(y_test, y_pred)
    ll = log_loss(y_test, rf.predict_proba(X_test))
    # Compute choice-set accuracy
    test_groups = groups.iloc[test_idx].values
    df_test = pd.DataFrame({'Obs_ID': test_groups, 'true_choice': y_test.values, 'pred_prob': y_proba})
    correct_choice_count = 0
    for obs_id, grp in df_test.groupby('Obs_ID'):
        # Identify the alternative with highest predicted probability in this observation
        idx_max = grp['pred_prob'].idxmax()
        chosen_flag = grp.loc[idx_max, 'true_choice']  # actual choice flag of that predicted route
        if chosen_flag == 1:
            correct_choice_count += 1
    choice_set_acc = correct_choice_count / df_test['Obs_ID'].nunique()
    # Store metrics and feature importances
    fold_metrics.append((acc, ll, choice_set_acc))
    feature_importances.append(rf.feature_importances_)
    print(f"Fold {fold}: Accuracy = {acc:.3f}, Log-Loss = {ll:.3f}, Choice-Set Accuracy = {choice_set_acc:.3f}")

# Compute average and std of metrics across folds
metrics_array = pd.DataFrame(fold_metrics, columns=['Accuracy', 'LogLoss', 'ChoiceSetAcc'])
avg_metrics = metrics_array.mean()
std_metrics = metrics_array.std()
print("\nCV Average Metrics:")
for metric in ['Accuracy', 'LogLoss', 'ChoiceSetAcc']:
    print(f"{metric}: {avg_metrics[metric]:.3f} ± {std_metrics[metric]:.3f}")


importances = np.vstack(feature_importances)
mean_imp = importances.mean(axis=0)
std_imp  = importances.std(axis=0)

# Top 10 features
top_idx = np.argsort(mean_imp)[::-1][:10]
top_features = np.array(features)[top_idx]

plt.figure(figsize=(8,6))
plt.barh(
    top_features[::-1],
    mean_imp[top_idx][::-1],
    xerr=std_imp[top_idx][::-1]
)
plt.xlabel("Mean Feature Importance")
plt.title("Top 10 Feature Importances (± Std)")
plt.tight_layout()
plt.show()